In [1]:
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
import numpy as np

In [2]:
DATASET_PATH = "./data/4th-cupoy-ml-100-marathon-finalexam/image_data"
IMAGE_SIZE = (224, 224, 3)
NUM_CLASSES = 5
BATCH_SIZE = 32
EPOCHS = 200
FREEZE_LAYERS = 25

In [3]:
train_datagen = ImageDataGenerator(rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode="nearest")

train_generator = train_datagen.flow_from_directory(DATASET_PATH + "/train",
                                                    target_size=(224, 224),
                                                    interpolation="bicubic",
                                                    class_mode="categorical",
                                                    shuffle=True,
                                                    batch_size=BATCH_SIZE)


def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    return lr

Found 2823 images belonging to 5 classes.


In [6]:
from keras.models import load_model

try:
    model = load_model("./saved_models/model_D101-103.h5")
    print("Model loaded successfully!")
except:
    print("Failed to load model!")
    resnet = ResNet50(include_top=False,
                     weights="imagenet",
                     input_tensor=None,
                     input_shape=IMAGE_SIZE)

    x = resnet.output
    x = Flatten()(x)
    x = Dropout(0.2)(x)

    output = Dense(NUM_CLASSES, activation="softmax", name="output")(x)

    model = Model(resnet.input, output)

    for layer in model.layers[:FREEZE_LAYERS]:
        layer.trainable = False
    for layer in model.layers[FREEZE_LAYERS:]:
        layer.trainable = True

    model.compile(optimizer=Adam(lr=lr_schedule(0)),
                  loss='categorical_crossentropy',
                  metrics=["accuracy"])

    print(model.summary())

    # 使用動態調整學習率
    lr_scheduler = LearningRateScheduler(lr_schedule)

    # 使用自動降低學習率 (當 validation loss 連續 5 次沒有下降時，自動降低學習率)
    lr_reducer = ReduceLROnPlateau(monitor="loss",
                                   factor=np.sqrt(0.1),
                                   cooldown=0,
                                   patience=5,
                                   min_lr=0.5e-6)
    # 設定 callbacks
    callbacks = [lr_reducer, lr_scheduler]

    model.fit(x=train_generator,
             steps_per_epoch = train_generator.samples // BATCH_SIZE,
             epochs=EPOCHS,
             callbacks=callbacks)

    model.save("./saved_models/model_D101-103.h5")

Model loaded successfully!


In [7]:
from keras.preprocessing import image
from os import walk, path
import pandas as pd

images =  walk(DATASET_PATH + "/test")

data = []

for root, dirs, files in images:
    for name in files:
        Path = path.join(root, name)
        img = image.load_img(Path, target_size=(224, 224))
        if img is None:
            continue
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        pred = model.predict(img)[0]
        top_inds = pred.argsort()[::-1][:5]
        data.append([name[:-4], list(pred).index(max(pred))])
        
result = pd.DataFrame(data, columns=["id", "flower_class"])
result.to_csv("Day101-103.csv", index=False)